In [1]:
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

In [2]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [3]:
class Bet:

    def __init__(self,K,T,observedPrice):
        self.K=K
        self.T=T
        self.observedPrice=observedPrice    

    def impliedVol(self,dynamics):
        # ignores dynamics.sigma, because this function solves for sigma.  

        if self.K != dynamics.S0:
            raise ValueError('Please assume the ATM case')
        if dynamics.r <= 0: 
            raise ValueError('Please assume r>0')

        if self.observedPrice is None: 
            raise ValueError('Contract price must be given')

        # fill this in.  You may use a root-finder, but if you see a way to code this without using a root-finder, 
        # then you are not required to use a root-finder.
            
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S0 / df
        C = self.observedPrice
        # Checking of these edge cases is _not_ required for full credit on this problem
        if C<0:
            return np.nan
        if C==0:
            return 0
        if C>=df:
            return np.nan 

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C
        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
        hi = dytry.sigma
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
        # In other words, the implied volatility is somewhere in the interval [lo,hi].

        impliedVol = bisect(lambda x: self.BSprice(dynamics.update_sigma(x))-C,lo,hi)

        return impliedVol

    def BSprice(self, dynamics):
        # Price a BINARY put
        # Ignores self.observedPrice if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S0*np.exp(dynamics.r*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d2 = np.log(F/self.K)/sd - sd/2
        return np.exp(-dynamics.r*self.T)*(1-norm.cdf(d2))
    

In [4]:
class EYcontract:

    def __init__(self,Klow,Kmid,Khigh,K0,T0,T1):
        self.Klow = Klow
        self.Kmid = Kmid
        self.Khigh = Khigh
        self.K0 = K0
        self.T0 = T0
        self.T1 = T1

In [5]:
class MC:

    def __init__(self,M,seed):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def randomLogreturn(self,dynamics,deltat):
        return (dynamics.r-dynamics.sigma**2/2)*deltat + dynamics.sigma*np.sqrt(deltat)*self.rng.normal(size=self.M)

    def price_EYcontract_GBM(self,contract,dynamics):
    
        S05 = dynamics.S0 * np.exp(self.randomLogreturn(dynamics,contract.T0))
        S10 = S05 * np.exp(self.randomLogreturn(dynamics,contract.T1-contract.T0))
        zeropayoff = np.zeros(self.M)
        payoffdiscounted = np.exp(-dynamics.r*contract.T1) * np.where((S10>contract.Khigh)|(S05>contract.K0), np.maximum(S10-contract.Kmid,zeropayoff), np.maximum(S10-contract.Klow,zeropayoff))
        price = np.mean(payoffdiscounted)
        standard_error = np.std(payoffdiscounted)/np.sqrt(self.M)

        return(price, standard_error)
        

## Problem 1

In [6]:
# your solution should allow for minor perturbations of the parameters, 
# but you may assume that r>0 and that S0=K 

p1dynamics=GBM(S0=88,r=0.03)

In [7]:
p1contract=Bet(K=88,T=1.0,observedPrice=0.5)  
#the 0.5 assumes that the contract pays $1 (or nothing).  
#If instead you assume it pays $20 million, then this observed price needs to be changed to $10 million.

In [8]:
impliedVol = p1contract.impliedVol(p1dynamics)
print(impliedVol)

0.28608478037494933


(1b) Binary put price $C = e^{-rT}N(-d_2) = e^{-rT}N\big(\frac{\sigma\sqrt{T}}{2}-\frac{\log(F/K)}{\sigma\sqrt{T}}\big)$

Vega $=\partial C/\partial\sigma = e^{-rT}N'(-d_2)\big(\frac{\sqrt{T}}{2} +\frac{\log(F/K)}{\sigma^2\sqrt{T}}\big)>0⟹$  Long vol.

## Problem 4

In [9]:
p4dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [10]:
p4contract = EYcontract(Klow=10,Kmid=11,Khigh=14,K0=12,T0=0.5,T1=1.0)

In [11]:
p4MC = MC(M=100000,seed=0) 

In [12]:
(price, standard_error) = p4MC.price_EYcontract_GBM(p4contract,p4dynamics)
print(price, standard_error)

2.561452804890789 0.019584331574493807
